In [1]:
import pandas as pd

# Read the file
file_path = 'Datasets/2_Delft.txt'

# Read the general parameters line
with open(file_path, 'r') as file:
    general_params = file.readline().strip()

# Read the rest of the file into a DataFrame
df = pd.read_csv(file_path, sep=' ', skiprows=1, header=None)
df.columns = ['Start_Horizontal', 'Start_Vertical', 'Finish_Horizontal', 'Finish_Vertical', 'Earliest_Start', 'Latest_Finish']

# Displaying the DataFrame
print("General Parameters:", general_params)
print("Delivery Data:")
df

General Parameters: 800 1000 100 300 25 25000
Delivery Data:


,Start_Horizontal,Start_Vertical,Finish_Horizontal,Finish_Vertical,Earliest_Start,Latest_Finish
0,395,43,296,955,244,3620
1,734,467,300,833,21919,23595
2,473,815,21,810,24428,24959
3,210,150,387,12,12646,13491
4,314,563,141,419,2744,3381
...,...,...,...,...,...,...
295,561,555,125,269,11932,13489
296,3,203,346,877,721,3756
297,606,724,154,616,20196,21068
298,463,972,343,532,13263,14196


In [2]:
import pandas as pd

def read_data(file_path):
    # Read the general parameters line
    with open(file_path, 'r') as file:
        general_params = list(map(int, file.readline().strip().split()))

    # Read the rest of the file into a DataFrame
    df = pd.read_csv(file_path, sep=' ', skiprows=1, header=None)
    df.columns = ['Start_Horizontal', 'Start_Vertical', 'Finish_Horizontal', 'Finish_Vertical', 'Earliest_Start', 'Latest_Finish']

    return general_params, df

def calculate_distance(point1, point2):
    """Calculate the Manhattan distance between two points."""
    return abs(point1[0] - point2[0]) + abs(point1[1] - point2[1])

def nearest_neighbor_route(df, sleds, T):
    routes = [[] for _ in range(sleds)]
    unvisited = df.index.tolist()

    for i in range(sleds):
        if unvisited:
            current_point = (df.loc[unvisited[0], 'Start_Horizontal'], df.loc[unvisited[0], 'Start_Vertical'])
            routes[i].append(current_point)
            unvisited.remove(unvisited[0])

            while unvisited:
                nearest_point, nearest_idx = None, None
                min_distance = float('inf')

                for idx in unvisited:
                    next_point = (df.loc[idx, 'Start_Horizontal'], df.loc[idx, 'Start_Vertical'])
                    distance = calculate_distance(current_point, next_point)
                    if distance < min_distance:
                        min_distance = distance
                        nearest_point = next_point
                        nearest_idx = idx

                if nearest_point and calculate_distance(current_point, nearest_point) + len(routes[i]) <= T:
                    current_point = nearest_point
                    routes[i].append(current_point)
                    unvisited.remove(nearest_idx)
                else:
                    break

    return routes

# Example usage
file_path = 'Datasets/2_Delft.txt'
general_params, df = read_data(file_path)
H, V, S, P, B, T = general_params
routes = nearest_neighbor_route(df, S, T)
for i, route in enumerate(routes):
    print(f"Route for Sled {i+1}: {route}")


Route for Sled 1: [(395, 43), (385, 37), (357, 20), (348, 7), (310, 10), (311, 1), (311, 37), (296, 47), (277, 50), (266, 66), (275, 87), (243, 102), (229, 110), (210, 150), (234, 186), (219, 215), (216, 217), (258, 224), (291, 207), (324, 207), (319, 195), (310, 163), (332, 100), (400, 136), (427, 128), (468, 106), (481, 80), (480, 43), (426, 66), (423, 12), (528, 3), (544, 27), (582, 50), (583, 59), (601, 45), (625, 46), (621, 39), (597, 94), (557, 123), (559, 126), (540, 111), (511, 139), (502, 138), (498, 128), (495, 147), (471, 167), (468, 170), (506, 215), (517, 208), (528, 267), (552, 276), (558, 292), (558, 342), (557, 342), (524, 323), (497, 303), (495, 283), (482, 285), (449, 270), (432, 268), (400, 309), (378, 301), (362, 356), (345, 370), (379, 411), (426, 438), (432, 467), (474, 470), (481, 487), (500, 484), (525, 464), (517, 426), (530, 416), (534, 379), (584, 384), (637, 404), (623, 425), (631, 440), (635, 475), (648, 494), (613, 505), (593, 508), (582, 461), (587, 566),